In [56]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Flatten, Conv2D
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import ModelCheckpoint,EarlyStopping
import tensorflow as tf

In [40]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3"
gpus = tf.config.experimental.list_logical_devices('GPU')
print(gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except:
        print('checking gpu error')
print('checking GPUs')

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config=tf.compat.v1.ConfigProto()
# config.gpu_options.visible_device_list= '0,1'
config.gpu_options.allow_growth=True
config.log_device_placement=True
# config.visible_device_list =2
sess=tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

[LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU'), LogicalDevice(name='/device:GPU:2', device_type='GPU'), LogicalDevice(name='/device:GPU:3', device_type='GPU')]
checking gpu error
checking GPUs
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:2 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:3 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:05:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: TITAN X (Pascal), pci bus id: 0000:06:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:2 -> device: 2, name: TITAN X (Pascal), pc

In [72]:
def _build_model():
    model_sub_in = Input(shape=(1, 7))
    # goalx,goaly, posx, posy, v, a, theta
    model_sub_flatten = Flatten()(model_sub_in)
    model_sub_mid = Dense(128, activation='relu', name='layer_sub_mid')(model_sub_flatten)
    model_sub_out = Dense(256, activation='elu', name='layer_sub_out')(model_sub_mid)
    model_sub = Model(model_sub_in, model_sub_out)

    model_veh_in = Input(shape=(3, 10, 2))
    # history traj veh
    model_veh_conv1 = Conv2D(128,(2,2),  activation='relu', name='layer_veh_conv1')(model_veh_in)
    model_veh_conv2 = Conv2D(256,(2,2),  activation='relu', name='layer_veh_conv2')(model_veh_conv1)
    model_veh_flatten = Flatten()(model_veh_conv2)
    model_veh_out = Dense(128,  activation='relu', name='layer_veh_out')(model_veh_flatten)
    model_veh = Model(model_veh_in, model_veh_out)
    
    model_ped_in = Input(shape=(3, 10, 2))
    # history traj ped
    model_ped_conv1 = Conv2D(128,(2,2),  activation='relu', name='layer_ped_conv1')(model_ped_in)
    model_ped_conv2 = Conv2D(256,(2,2),  activation='relu', name='layer_ped_conv2')(model_ped_conv1)
    model_ped_flatten = Flatten()(model_ped_conv2)
    model_ped_out = Dense(128,  activation='elu', name='layer_ped_out')(model_ped_flatten)
    model_ped = Model(model_ped_in, model_ped_out)
    
    
    model_cyc_in = Input(shape=(3, 10, 2))
    # history traj cyc
    model_cyc_conv1 = Conv2D(128,(2,2),  activation='relu', name='layer_cyc_conv1')(model_cyc_in)
    model_cyc_conv2 = Conv2D(128,(2,2),  activation='relu', name='layer_cyc_conv2')(model_cyc_conv1)
    model_cyc_flatten = Flatten()(model_cyc_conv2)
    model_cyc_out = Dense(128,  activation='elu', name='layer_cyc_out')(model_cyc_flatten)
    model_cyc = Model(model_cyc_in, model_cyc_out)
    
    
    
    model_edge_in = Input(shape=(1, 9))
    # edges connected to subject
    model_edge_flatten = Flatten()(model_edge_in)
    model_edge_mid = Dense(128,  activation='relu', name='layer_edge_mid')(model_edge_flatten)
    model_edge_out = Dense(256,  activation='relu', name='layer_edge_output')(model_edge_mid)
    model_edge = Model(model_edge_in, model_edge_out)


    concatenated = concatenate([model_sub_out, model_veh_out,model_ped_out,model_cyc_out,model_edge_out])
    out = Dense(1800, activation='softmax', name='output_layer')(concatenated)
    
    # define the deep RL model below.
    
    # 
    merged_model = Model([model_sub_in, model_veh_in,model_ped_in,model_cyc_in,model_edge_in], out)
    return merged_model


In [77]:
merged_model = _build_model()
merged_model.compile(loss='binary_crossentropy', optimizer='adagrad', 
metrics=['accuracy'])
merged_model.summary()

checkpoint = ModelCheckpoint('weights.h5', monitor='val_acc',
                             save_best_only=True, verbose=2)
early_stopping = EarlyStopping(monitor="val_loss", patience=10)


Model: "model_96"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_93 (InputLayer)           [(None, 3, 10, 2)]   0                                            
__________________________________________________________________________________________________
input_94 (InputLayer)           [(None, 3, 10, 2)]   0                                            
__________________________________________________________________________________________________
input_95 (InputLayer)           [(None, 3, 10, 2)]   0                                            
__________________________________________________________________________________________________
input_92 (InputLayer)           [(None, 1, 7)]       0                                            
___________________________________________________________________________________________

In [78]:
#synthetic data
sub = np.random.rand(320000, 1,7)
veh =np.random.rand(320000,3,10,2)
ped =np.random.rand(320000,3,10,2)
cyc =np.random.rand(320000,3,10,2)
edge =np.random.rand(320000,1,9)
y=np.random.randint(0,20000,size=1800)
edge.shape

(320000, 1, 9)

In [79]:

merged_model.fit([sub, veh, ped, cyc, edge], y=y, batch_size=32, epochs=10,
             verbose=1, validation_split=0.1, shuffle=True
# callbacks=[early_stopping, checkpoint]
                )

Epoch 1/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6199 - accuracy: 1.0417e-05 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 2/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6173 - accuracy: 0.0000e+00 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 3/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6171 - accuracy: 6.9444e-06 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 4/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6169 - accuracy: 3.4722e-06 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 5/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6168 - accuracy: 2.4306e-05 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 6/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6168 - accuracy: 1.3889e-05 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 7/10
9000/9000 [========================